In [48]:
import json
import pandas as pd
from collections import defaultdict

In [26]:
issues_df = pd.read_json("Devel/robojackets-stats/robocup-issues1.json")

In [39]:
issues_df.shape

(60, 22)

In [37]:
issues_df.head()

,id,node_id,number,title,user,labels,state,locked,assignee,assignees,...,created_at,updated_at,closed_at,author_association,active_lock_reason,pull_request,body,reactions,timeline_url,performed_via_github_app
0,1042930255,PR_kwDOAO78Gc4uAiD9,1742,Two robots passing,"{'login': 'Yudai-8-om', 'id': 53668296, 'node_...",[],open,False,None,[],...,2021-11-02 23:15:39+00:00,2021-11-02 23:15:39+00:00,NaT,CONTRIBUTOR,NaN,{'url': 'https://api.github.com/repos/RoboJack...,## Description\r\nbasic passing,{'url': 'https://api.github.com/repos/RoboJack...,https://api.github.com/repos/RoboJackets/roboc...,NaN
1,1034582888,PR_kwDOAO78Gc4tmQXj,1741,foreach in all plays,"{'login': 'hdemusg', 'id': 10521124, 'node_id'...",[],open,False,None,[],...,2021-10-25 01:32:03+00:00,2021-11-02 03:02:40+00:00,NaT,CONTRIBUTOR,NaN,{'url': 'https://api.github.com/repos/RoboJack...,## Description\r\nReplaced repetitive statemen...,{'url': 'https://api.github.com/repos/RoboJack...,https://api.github.com/repos/RoboJackets/roboc...,NaN
2,1030848119,PR_kwDOAO78Gc4taVNu,1739,Role stickiness,"{'login': 'Yudai-8-om', 'id': 53668296, 'node_...",[],open,False,None,[],...,2021-10-20 00:21:02+00:00,2021-11-02 03:23:14+00:00,NaT,CONTRIBUTOR,NaN,{'url': 'https://api.github.com/repos/RoboJack...,## Description\r\nApplied role stickiness\r\n\...,{'url': 'https://api.github.com/repos/RoboJack...,https://api.github.com/repos/RoboJackets/roboc...,NaN
3,1030845227,PR_kwDOAO78Gc4taUmf,1738,"working action, untested server","{'login': 'kfu02', 'id': 30275369, 'node_id': ...",[],open,False,None,[],...,2021-10-20 00:14:30+00:00,2021-11-03 00:54:34+00:00,NaT,CONTRIBUTOR,NaN,{'url': 'https://api.github.com/repos/RoboJack...,temp again to see CI,{'url': 'https://api.github.com/repos/RoboJack...,https://api.github.com/repos/RoboJackets/roboc...,NaN
4,1007534072,PR_kwDOAO78Gc4sSMjX,1733,Removed HML and replaced with arbitrary floats,"{'login': 'anshulm07', 'id': 59479669, 'node_i...",[],open,False,None,[],...,2021-09-26 23:20:58+00:00,2021-11-01 00:39:28+00:00,NaT,NONE,NaN,{'url': 'https://api.github.com/repos/RoboJack...,Removed HML and replaced all uses of HML in ta...,{'url': 'https://api.github.com/repos/RoboJack...,https://api.github.com/repos/RoboJackets/roboc...,NaN


In [36]:
issues_df = issues_df.drop(columns=["url","repository_url","labels_url","comments_url","events_url","html_url"])

In [59]:
title_word_counts = defaultdict(int)

In [60]:
def get_title_word_counts(title_text, word_count_dict=title_word_counts):
    split_words = title_text.split(" ")
    #print(split_words)
    for word in split_words:
        word_count_dict[word.lower()] += 1
    return

In [61]:
#get_title_word_counts("Two robots passing", title_word_counts)

In [62]:
issues_df["title"].apply(get_title_word_counts)

['Two', 'robots', 'passing']
['foreach', 'in', 'all', 'plays']
['Role', 'stickiness']
['working', 'action,', 'untested', 'server']
['Removed', 'HML', 'and', 'replaced', 'with', 'arbitrary', 'floats']
['[Draft]', 'ERsim', 'submoduling']
['[TODO]', 'Remove', 'goalie', 'brick', 'mode']
['[DRAFT]', 'Goalie', 'utility', 'function']
['[TODO]', 'Figure', 'out', 'which', 'QT', 'package', 'is', 'breaking', 'VNC', 'on', 'comp', 'docker.']
['Debug', 'drawing', 'assumes', 'polygon', 'vertices', 'will', 'be', 'sent', 'in', 'counterclockwise', 'order']
['Properly', 'handle', 'ball', 'occlusion', 'in', 'vision']
['[TODO]', 'Handle', 'infinite', 'cost', 'in', 'role', 'assignment']
['[TODO]', 'Delete', 'shoot', 'skill']
['[TODO]', 'Combine', 'direct/indirect']
['[TODO]', 'Fix', 'hardcoding', 'in', 'gameplay']
['[TODO]', 'Break', 'out', 'goal', 'zone', 'geometry', 'calculations', 'from', 'gameplay_node.py']
['Evaluate', 'QoS']
['made', 'static', 'attack', 'play', 'for', 'hardware', 'challenge']
['Triang

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
53    None
54    None
55    None
56    None
57    None
58    None
59    None
Name: title, dtype: object

In [63]:
title_word_counts

defaultdict(int,
            {'two': 1,
             'robots': 2,
             'passing': 2,
             'foreach': 1,
             'in': 8,
             'all': 2,
             'plays': 1,
             'role': 3,
             'stickiness': 1,
             'working': 1,
             'action,': 1,
             'untested': 1,
             'server': 2,
             'removed': 1,
             'hml': 1,
             'and': 4,
             'replaced': 1,
             'with': 2,
             'arbitrary': 1,
             'floats': 1,
             '[draft]': 2,
             'ersim': 1,
             'submoduling': 1,
             '[todo]': 12,
             'remove': 2,
             'goalie': 2,
             'brick': 1,
             'mode': 1,
             'utility': 1,
             'function': 2,
             'figure': 2,
             'out': 3,
             'which': 1,
             'qt': 1,
             'package': 1,
             'is': 1,
             'breaking': 1,
             'vnc': 1,
      